# Reproduce the results presented in "CRISPR-HAWK: Haplotype- and Variant-Aware Guide Design Toolkit for CRISPR-Cas"

In [ ]:
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import pandas as pd
import numpy as np

import subprocess
import random
import pysam
import os

from time import time
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties
from matplotlib.colors import LinearSegmentedColormap

## Introduction

CRISPR-HAWK is a comprehensive and scalable framework for designing guide RNAs 
(gRNAs) and evaluating the impact of genetic variation on CRISPR-Cas on-target 
activity. Developed as an offline, user-friendly command-line tool, CRISPR-HAWK 
integrates large-scale human variation datasets, including the 1000 Genomes Project, 
the Human Genome Diversity Project (HGDP), and gnomAD, with orthogonal 
genomic annotations to systematically prioritize gRNAs targeting regions of interest.

The framework is Cas-agnostic and supports a broad range of nucleases, such as 
Cas9, SaCas9, and Cpf1 (Cas12a), while also allowing full customization of PAM 
sequences and guide lengths. This flexibility ensures compatibility with emerging 
CRISPR technologies and enables users to tailor gRNA design to specific experimental 
needs.

CRISPR-HAWK incorporates both single-nucleotide variants (SNVs) and small 
insertions and deletions (indels), and it natively handles individual- and 
population-specific haplotypes. This makes it particularly suitable for personalized 
genome editing as well as population-scale analyses. The workflow, from variant-aware 
preprocessing to gRNA discovery, is fully automated, generating ranked candidate 
gRNAs, annotated target sequences, and publication-ready visualizations.

Thanks to its modular architecture, CRISPR-HAWK can be seamlessly integrated with 
downstream tools such as CRISPRme or CRISPRitz for comprehensive off-target prediction 
and follow-up analysis of prioritized guides.

This notebook reproduce the results presented in |||**add paper-link**|||.

## Download data

### Downloading Genetic Variation Datasets (1000 Genomes, HGDP, gnomAD)

CRISPR-HAWK supports large-scale genetic diversity analyses by integrating variation 
from several major population genomics resources. This section provides instructions 
for downloading the VCF files required to reproduce the results presented in the paper.

**Overview of Supported Datasets**

- 1000 Genomes Project (Phase 3)
  <br>2,504 individuals from 26 global populations; whole-genome sequencing at ~30×.

- Human Genome Diversity Project (HGDP)
  <br>929 individuals from globally diverse populations; high-coverage WGS.

- gnomAD (v4.1)
  <br>Population-scale aggregated variation from ~76,000 genomes.

In [ ]:
# define datasets urls
url_hgdp = (
    "https://ngs.sanger.ac.uk/production/hgdp/hgdp_wgs.20190516/" 
    "hgdp_wgs.20190516.full.chr{}.vcf.gz"
)
url_1kgp = (
    "ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/"
    "1000_genomes_project/release/20190312_biallelic_SNV_and_INDEL/" 
    "ALL.chr{}.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz"
)
url_gnomad = (
    "https://storage.googleapis.com/gcp-public-data--gnomad/release/4.1/vcf/" 
    "genomes/gnomad.genomes.v4.1.sites.chr{}.vcf.bgz"
)
variants = {"HGDP": url_hgdp, "1000G": url_1kgp, "GNOMAD": url_gnomad}

# define chromosomes
chroms = [2, 3, 7, 11]

# download files
for dataset, url in variants.items():
    vcfdir = os.path.join("vcf", dataset)
    if dataset == "GNOMAD":
        vcfdir = os.path.join(vcfdir, "raw")
    os.makedirs(vcfdir, exist_ok=True)  # create dataset folder 
    for c in chroms:        
        # download VCF and index
        ! wget -P {vcfdir} {url.format(c)}
        ! wget -P {vcfdir} {url.format(c)}.tbi

Unlike the 1000 Genomes and HGDP callsets, the gnomAD VCFs contain allele frequency 
information only and do not provide individual-level genotype data. Since 
CRISPR-HAWK requires genotypes to reconstruct haplotypes and perform variant-aware 
gRNA discovery, gnomAD VCFs must first be converted into a compatible, pseudo-genotyped 
format. To enable this, we use the CRISPR-HAWK VCF converter, which generates a 
CRISPR-HAWK–ready VCF while preserving population-level allele distributions.

In [ ]:
# check gnomad folder exists
gnomad_dir = "vcf/GNOMAD"
gnomad_raw_dir = os.path.join(gnomad_dir, "raw")
assert os.path.isdir(gnomad_raw_dir)

# create converted VCFs folder
gnomad_gt_dir = os.path.join(gnomad_dir, "genotype")
os.makedirs(gnomad_gt_dir, exist_ok=True)

# convert gnoamd VCFs using crisprhawk (it may take some time)
! crisprhawk convert-gnomad-vcf -d {gnomad_raw_dir} -o {gnomad_gt_dir}


### Downloading the hg38 Reference Genome

CRISPR-HAWK requires a reference genome to extract genomic contexts, evaluate 
variant-aware target sequences, and correctly map gRNA target sites. In this 
section, we download the primary assembly FASTA filesfrom the Genome Reference 
Consortium–maintained repositories.

In [ ]:
# define chromosomes
chroms = [2, 3, 7, 11]

# create genome folder
genome_dir = "genome"
os.makedirs(genome_dir, exist_ok=True)

# define genome url
url_ucsc = (
    "https://hgdownload.soe.ucsc.edu/goldenpath/hg38/chromosomes/chr{}.fa.gz"
)

# Download and unzip FASTA file for each chromosome
for c in chroms:
    print(f"Downloading FASTA for chromosome {c}")
    ! wget -nc -P {genome_dir} {url_ucsc.format(c)}
    ! gunzip -f {genome_dir}/chr{c}.fa.gz

### Creating BED Files for the Analyzed Regions

CRISPR-HAWK requires BED files to define the genomic intervals where gRNA discovery 
and variant-aware analysis will be performed. Each BED file specifies one or more 
regions of interest in the standard 3-column BED format:
```
chrom   start   end
```

Coordinates must reference the hg38 genome assembly (or whichever reference you 
downloaded in the previous step).

In [ ]:
# create region folder
regions_dir = "regions"
os.makedirs(regions_dir, exist_ok=True)

# define regions
regions = {
    "BCL11A": ["chr2", 60495215, 60496479],
    "EMX1":   ["chr2", 72932853, 72934853],
    "CCR5_1":   ["chr3", 46372138, 46374138],
    "CCR5_2":   ["chr3", 46372162, 46374162],
    "TRBC1":  ["chr7", 142791004, 142793004],
    "TRBC2":  ["chr7", 142800351, 142802350],
    "HBB_1":    ["chr11", 5225803, 5227803],
    "HBB_2":    ["chr11", 5225967, 5227967],
    "HBG2_CAS9":   ["chr11", 5252879, 5256879],
    "HBG1_CAS9":   ["chr11", 5248955, 5250955],
    "HBG2_CPF1":   ["chr11", 5253874, 5255874],
    "HBG1_CPF1":   ["chr11", 5248950, 5250950],
    "FANCF":  ["chr11", 22624785, 22626785],
}

# create bed files 
for gene, (chrom, start, end) in regions.items():
    bed_fname = os.path.join(regions_dir, f"{gene}.bed")
    with open(bed_fname, mode="w") as f:
        f.write(f"{chrom}\t{start}\t{end}\n")


## Variant-Aware gRNA Retrieval on Defined Regions

With the reference genome, variant datasets, and BED files prepared, CRISPR-HAWK
can now retrieve all candidate gRNAs within the specified regions while accounting
for genetic variation across populations and individuals. This step is central to 
CRISPR-HAWK’s design philosophy: guide discovery must be variant-aware, ensuring 
that both reference and haplotype-specific target sequences are evaluated.

**What This Step Does**

For each genomic interval listed in the BED file(s), CRISPR-HAWK:

- Extracts the reference sequence from hg38.

- Applies all relevant variants (SNVs and indels) from the loaded dataset,
including 1000G, HGDP, or converted gnomAD, to reconstruct individual, and population-specific haplotypes.

- Scans both the reference and haplotype sequences to identify all gRNAs that match the user-defined:

    - PAM sequence (e.g., NGG, NAA, TTTV, …)

    - guide length

    - nuclease type (Cas9, SaCas9, Cpf1/Cas12a, etc.)

- Reports each gRNA along with:

    - its exact reference and alternative alleles

    - per-haplotype presence/absence

    - the samples in which the target site is altered

    - summary metrics for prioritization


In [ ]:
# define target regions
targets = [
    "BCL11A",
    "EMX1",
    "CCR5_1",
    "CCR5_2",
    "TRBC1",
    "TRBC2",
    "FANCF",
    "HBB_1",
    "HBB_2",
    "HBG1_CAS9",
    "HBG2_CAS9",
    "HBG1_CPF1",
    "HBG1_CPF1",
]

# define genetic variants datasets
datasets = ["1000G", "HGDP", "GNOMAD"]

# define pams
pams = ["NGG", "TTTV"] # Cas9, Cas12
guide_lens = [20, 23]
thread = 16

# define folders 
genome_dir = "genome"
variants_dir = "vcf"
regions_dir = "regions"
results_dir = "results"

# create results folder
os.makedirs(results_dir, exist_ok=True)

# run guide design with crisprhawk
for dataset in datasets:
    vcfdir = os.path.join(variants_dir, dataset)
    if dataset == "GNOMAD":
        vcfdir = os.path.join(vcfdir, "genotype")
    results_dataset = os.path.join(results_dir, dataset)
    os.makedirs(results_dataset, exist_ok=True)  # e.g. results/1000G 
    for target in targets:
        target_region = os.path.join(regions_dir, f"{target}.bed")
        pam = pams[1] if target.endswith("_CPF1") else pams[0]
        results_target = os.path.join(results_dataset, target)  # e.g. results/1000G/BCL11A
        guidelen = 20 if pam == "NGG" else 23
        crisprhawk_cmd = (
            "crisprhawk search " 
            f"-f {genome_dir} " 
            f"-r {target_region} "
            f"-v {vcfdir} "
            f"-p {pam} "
            f"-g {guidelen} "
            f"--haplotype-table "
            "--threads 16 "
            f"-o {results_target}"
        )
        if pam == "TTTV":
            crisprhawk_cmd += " --right"  # pam upstream spacer
        print(f"Running search on {dataset} for target {target}")
        subprocess.call(crisprhawk_cmd, shell=True)

## Results Analysis and Visualization

In this section, we analyze and visualize the impact of genetic variation on gRNA 
design and activity across a curated set of clinically and experimentally relevant 
CRISPR targets. The analysis focuses on how population-level and individual-specific 
variants affect both gRNA sequence composition and expected on-target efficiency, 
highlighting differences between reference-designed guides and their alternative, 
haplotype-derived counterparts.

For each target region, we first quantify how genetic variation alters the landscape 
of candidate gRNAs. Specifically, we classify retrieved guides into four categories 
and summarize them using pie charts:
- gRNAs matching the reference sequence
- gRNAs with variants in the spacer region
- gRNAs with variants affecting only the PAM
- gRNAs with variants in both the spacer and the PAM

This provides an immediate overview of how frequently genetic variants modify 
targetable sequences across different loci. 

Next, we assess the functional consequences of these sequence differences. Using 
dot plots, we compare:
- The predicted on-target efficiency of reference gRNAs versus their alternative 
  versions found on variant-defined haplotypes
- The residual on-target activity of reference gRNAs when applied to alternative 
  haplotypes carrying sequence mismatches

These analyses capture both gain and loss of activity induced by genetic variation 
and enable a fine-grained comparison between reference and variant-aware gRNA designs.

All analyses are performed independently for the following target regions:
- BCL11A +58 Erythroid enhancer
- EMX1
- CCR5 (two independent target sites)
- TRBC1
- TRBC2
- FANCF
- HBB (two independent target sites)
- HBG1 and HBG2 (Cas9)
- HBG1 (Cpf1/Cas12a)

For Cpf1-based targets, residual on-target activity is not evaluated, as the 
analysis is specific to Cas9-mediated spacer–PAM interactions.

The analyses integrate variation from 1000 Genomes, HGDP, and gnomAD datasets. 
In particular, for the sg1617 guide targeting the BCL11A erythroid enhancer, we 
perform an in-depth follow-up analysis: for each alternative gRNA sequence generated 
by gnomAD variants, we run CRISPRme (using 1000G + HGDP genetic variants) to 
evaluate guide specificity genome-wide. This allows us to compare how genetic 
variation simultaneously affects on-target activity and off-target risk, 
providing a comprehensive assessment of guide performance in a population-aware 
context.

### Assigning Sample Support to Candidate gRNAs

As a first step in the results analysis, we quantify how widely each candidate 
gRNA is supported across individuals in each variant datasets. For every gRNA 
retrieved in the previous step, we compute the number of samples carrying the 
exact gRNA sequence in their reconstructed haplotypes.

These sample support counts form the basis for all downstream analyses in this 
section, including the evaluation of efficiency differences between reference 
and alternative guides, and reference on-target residual activity on alternative
haplotypes. 

For datasets providing individual-level genotypes (e.g., 1000 Genomes and HGDP), 
sample support is directly derived from haplotype reconstruction.

In [ ]:
SAMPLESNUM = {"1000G": 2504, "HGDP": 929, "GNOMAD": 76215}

def _compute_id(chrom, start, stop, strand):
    return f"{chrom}_{start}_{stop}_{strand}"

def compute_samples_num(df, samplesnum):
    assert "samples" in df.columns.tolist()
    # count samples carrying alternative grnas
    df["n_samples"] = df["samples"].apply(
        lambda x: len(str(x).split(",")) if pd.notna(x) and x!= "REF" else None
    )  
    # compute guide id for each grna
    df["guide_id"] = df.apply(
        lambda x: _compute_id(x["chr"], x["start"], x["stop"], x["strand"]), axis=1
    )
    # sum samples carrying alternative guides
    sum_alternative = df[df["samples"] != "REF"].groupby("guide_id")["n_samples"].sum()
    # retrieve number of samples for reference grnas
    refmask = df["samples"] == "REF"
    df.loc[refmask, "n_samples"] = samplesnum - df.loc[refmask, "guide_id"].map(sum_alternative).fillna(0)
    return df

For gnomAD-based analyses, individual genotypes are not directly available. In 
this case, CRISPR-HAWK reports the number of populations carrying each gRNA rather
than explicit sample counts. To enable downstream analyses requiring sample-level 
support, we post-process CRISPR-HAWK reports by annotating them with gnomAD 
carrier counts using pysam. This procedure estimates the number of carriers for
each variant allele underlying a gRNA and propagates this information to the 
guide level.

These sample support estimates are then used consistently with the other datasets 
to classify gRNAs and to perform comparative analyses across reference and 
variant-derived guides.

In [ ]:
# gnomad-only post-processing (NB: may require hours to run)
def _retrieve_position(variant_id):
    return int(variant_id.split("-")[1])

def extract_variant_positions(df):
    # retrieve position of each variant
    variant_ids = df["variant_id"].dropna().tolist()
    return {_retrieve_position(v) for vs in variant_ids for v in vs.split(",")}

def get_relevant_variants(vcf_fname, chrom, positions):
    print(f"Scanning VCF for {len(positions)} positions on {chrom}")
    vcf = pysam.VariantFile(vcf_fname)  # load vcf
    variants, matches = [], set()

    # requires bgzipped vcf with tbi index
    for r in tqdm(vcf.fetch(chrom), desc="Scanning VCF", unit="variants"):
        if r.pos not in positions:
            continue
    
        ac = r.info.get("AC")  # read allele count
        if ac is None:
            continue

        nhomalt = r.info.get("nhomalt", 0)  # count number of homozygous for alternative

        # iterate on each alternative allele
        for i, alt in enumerate(r.alts):
            # scalar vs per-allele fields for AC and nhomalt
            if isinstance(ac, (list, tuple)):
                ac_ = ac[i] if i < len(ac) else None
            else:
                ac_ = ac
            if ac_ is None:
                continue

            if isinstance(nhomalt, (list, tuple)):
                nhomalt_ = nhomalt[i] if i < len(nhomalt) else 0
            else:
                nhomalt_ = nhomalt

            # compute number of heterozygous samples
            n_het = ac_ - 2 * nhomalt_
            n_total = nhomalt_ + n_het

            # add variants and matched position
            variant_id = f"{r.chrom}-{r.pos}-{r.ref}/{alt}"
            variants.append(
                {
                    "variant_id": variant_id,
                    "chrom": r.chrom,
                    "pos": r.pos,
                    "ref": r.ref,
                    "alt": alt,
                    "AC": ac_,
                    "n_hom": nhomalt_,
                    "n_het": n_het,
                    "n_samples": n_total
                }
            )
            matches.add(r.pos)
    print(f"Matched {len(matches)} of {len(positions)} positions")
    return pd.DataFrame(variants)

def build_samples_dict(df_variants):
    # map variant IDs to total sample carrier counts
    df_sums = df_variants. \
        groupby("variant_id", sort=False)["n_samples"]. \
        sum(). \
        reset_index()
    return df_sums.set_index("variant_id")["n_samples"].to_dict()

def get_samples_total(variant_ids, samples_dict):
    if pd.isna(variant_ids):
        return "NA"
    total = sum(
        int(samples_dict[v]) 
        for v in variant_ids.split(",") 
        if v in samples_dict
    )
    return str(total) if total else "NA"

def annotate_gnomad_report(vcf_path, chrom, report_path):
    print("Loading CRISPR-HAWK report...")
    report_df = pd.read_csv(report_path, sep="\t")
    print(f"Loaded {len(report_df)} rows.")

    print("Extracting variant positions...")
    positions = extract_variant_positions(report_df)
    if not positions:
        print("No valid variant positions found. Exiting.")
        return report_df

    start = time()
    df_variants = get_relevant_variants(vcf_path, chrom, positions)

    # Write carriers file as COMMA-separated 
    variant_out = f"variants_carriers_{chrom}_pysam.tsv"
    df_variants.to_csv(variant_out, index=False)
    print(f"Saved variant carrier data to {variant_out} in {time() - start:.2f} seconds")

    # Build samples dict from grouped sums
    samples_dict = build_samples_dict(df_variants)

    # Annotate the report
    tqdm.pandas(desc="Annotating report")
    report_df["n_samples"] = report_df["variant_id"]. \
        progress_apply(lambda x: get_samples_total(x, samples_dict))
    output_path = f"{os.path.splitext(report_path)[0]}_samples_pysam.tsv"
    report_df.to_csv(output_path, sep="\t", index=False)
    print(f"\nDone. Annotated report saved to: {output_path}")

    return report_df

Now that we defined the functions to quantify how widely each candidate 
gRNA is supported across individuals in each variant datasets, we proceed by 
annotating BCL11A guide candidates.

In [ ]:
report_fname = "crisprhawk_guides__chr2_60495215_60496479_NGG_20.tsv"
report_fname_samples = "crisprhawk_guides__chr2_60495215_60496479_NGG_20_samples.tsv"

# 1000G 
bcl11a_1000G = pd.read_csv(f"results/1000G/BCL11A/{report_fname}", sep="\t")
bcl11a_1000G = compute_samples_num(bcl11a_1000G, SAMPLESNUM["1000G"])
bcl11a_1000G.to_csv(f"results/1000G/BCL11A/{report_fname_samples}", sep="\t", index=False)


# HGDP
bcl11a_HGDP = pd.read_csv(f"resuts/HGDP/BCL11A/{report_fname}", sep="\t")
bcl11a_HGDP = compute_samples_num(bcl11a_HGDP, SAMPLESNUM["HGDP"])
bcl11a_HGDP.to_csv(f"results/HGDP/BCL11A/{report_fname_samples}.tsv", sep="\t", index=False)

# gnomAD  -- this step takes several hours to complete
vcf_GNOMAD = "vcf/GNOMAD/raw/gnomad.genomes.v4.1.sites.chr2.vcf.bgz"
report_GNOMAD = f"results/GNOMAD/BCL11A/{report_fname}.tsv"
bcl11a_GNOMAD = annotate_gnomad_report(vcf_GNOMAD, "chr2", report_GNOMAD)

bcl11a_GNOMAD = compute_samples_num(bcl11a_GNOMAD, SAMPLESNUM["GNOMAD"])
bcl11a_GNOMAD.to_csv(f"results/GNOMAD/BCL11A/{report_fname_samples}.tsv", sep="\t", index=False)

## Types of Retrieved Guide Candidates

Here we classify all retrieved guide candidates according to their relationship 
with genetic variation. Each guide is assigned to one of the following categories:

- Reference Guides: guides fully supported by the reference genome and unaffected 
    by observed variants.

- PAM and Spacer Alternative Guides: guides that arise due to the presence of 
    genetic variants creating new protospacer or PAM sequences.

- PAM Alternative Guides: guides that arise due to the presence of 
    genetic variants creating new PAM sequences.

- Spacer Alternative Guides: guides that arise due to the presence of 
    genetic variants creating new protospacer sequences.

The pie charts below summarize the relative proportion of each guide type among 
all retrieved candidates. This visualization provides an intuitive overview of 
how much of the guide space would be missed or mischaracterized by a reference-only 
approach and highlights the contribution of variant- and haplotype-aware guide 
discovery.

In [ ]:
LABELS = [
    "Reference Guides",
    "Spacer Alternative Guides",
    "PAM Alternative Guides",
    "PAM and Spacer Alternative Guides"
]

def compute_guide_id_piechart(chrom, start, stop, strand, sgRNA_sequence, pam):
    return f"{chrom}_{start}_{stop}_{strand}_{sgRNA_sequence}_{pam}"


def has_lowercase(s):
    return any(c.islower() for c in str(s))


def retrieve_pie_chart_data(df):
    df["guide_id"] = df.apply(
        lambda x: compute_guide_id_piechart(
            x["chr"], x["start"], x["stop"], x["strand"], x["sgRNA_sequence"], x["pam"]
        ), 
        axis=1
    )
    df["category"] = "Unclassified"  # default

    # remove duplicate rows (same site and spacer+pam, different haplotype and scores)
    df = df.drop_duplicates(subset="guide_id")

    # category 1: Reference Guides
    df.loc[df["origin"] == "ref", "category"] = "Reference Guides"

    # category 2: PAM and Spacer Alternative Guides
    mask_non_ref = df["origin"] != "ref"
    condition2 = mask_non_ref & df["sgRNA_sequence"].apply(has_lowercase) & df["pam"].apply(has_lowercase)
    df.loc[condition2, "category"] = "PAM and Spacer Alternative Guides"

    # category 3: PAM Alternative Guides
    condition3 = mask_non_ref & df["pam"].apply(has_lowercase) & ~df["sgRNA_sequence"].apply(has_lowercase)
    df.loc[condition3, "category"] = "PAM Alternative Guides"

    # Category 4: Spacer Alternative Guides
    condition4 = mask_non_ref & df["sgRNA_sequence"].apply(has_lowercase) & ~df["pam"].apply(has_lowercase)
    df.loc[condition4, "category"] = "Spacer Alternative Guides"

    unclassified_count = (df["category"] == "Unclassified").sum()
    assert unclassified_count == 0, f"{unclassified_count} guides remain unclassified!"

    return [df["category"].value_counts().get(cat, 0) for cat in LABELS]


def piechart(ax, data, title):
    # colors for each category
    colors = ["#aaa1cdff", "#92c1deff", "#ef86bdff", "#b8d8c8"]
    explode = (0, 0, 0.05, 0.1)

    wedges, texts, autotexts = ax.pie(
        data,
        explode=explode,
        colors=colors,
        autopct='%1.2f%%',
        shadow=False,
        startangle=140,
        textprops={"fontsize": 16},
        pctdistance=1.1
    )

    ax.set_title(title, fontsize=20)
    ax.axis("equal")

Once defined the functions to plot our guide piechart, we display the gRNAs 
categories proportion in the three datasets on BCL11A enhancer region.

In [ ]:
report_fname = "crisprhawk_guides__chr2_60495215_60496479_NGG_20.tsv"
report_fname_samples = "crisprhawk_guides__chr2_60495215_60496479_NGG_20_samples.tsv"

bcl11a_1000G = pd.read_csv(f"results/1000G/BCL11A/{report_fname_samples}", sep="\t")
bcl11a_HGDP = pd.read_csv(f"results/HGDP/BCL11A/{report_fname_samples}", sep="\t")
bcl11a_GNOMAD = pd.read_csv(f"results/GNOMAD/BCL11A/{report_fname_samples}", sep="\t")

In [ ]:
datasets = ["1000G", "HGDP", "GNOMAD"]
data_report_map = {"1000G": bcl11a_1000G, "HGDP": bcl11a_HGDP, "GNOMAD": bcl11a_GNOMAD}

f, axes = plt.subplots(1, 3, figsize=(30, 10))

for ax, dataset in zip(axes, datasets):
    title = f"BCL11A +58 Erythroid Enhancer - {dataset}"

    # retrieve dataset-specific dataframe
    pie_data = retrieve_pie_chart_data(data_report_map[dataset])
    piechart(ax, pie_data, title)

# single shared legend
f.legend(LABELS, loc="lower center", ncol=4, fontsize=16, frameon=False)

plt.tight_layout()
plt.show()

## Guides On-Target Efficiency and Variant Effect on Alternative On-Targets 

Here we assess the predicted on-target efficiency of the retrieved guide candidates
and investigate how genetic variation influences the presence and scoring of 
alternative on-targets.

For each guide, CRISPR-HAWK computes an on-target efficiency score based on its 
spacer and PAM sequence. When variants alter either the spacer, the PAM, or both, 
alternative guide configurations may emerge at the same genomic locus. These 
alternatives can differ substantially in predicted efficiency compared to the 
reference guide.

The following analyses focus on:

- Comparing the efficiency of reference guides against their alternatives

- Assessing whether genetoc variants may modulate the expected likelihood of
    a guide designed on the reference to work properly on haplotype-specific
    on-target sites


Define global constants and special guide identifier

In [ ]:
SG1617 = "chr2_60495261_-"
SAMPLE_COL = "n_samples"

def compute_guide_id(chrom, start, strand):
    return f"{chrom}_{start}_{strand}"

Calculate score differences between reference and alternative guides

In [ ]:
def calculate_deltas(df, score_col):
    """Add delta columns comparing alt to ref scores for each guide."""
    df = df.copy()
    use_abs = score_col != "score_cfdon"
    
    def add_deltas(group):
        ref = group[group["origin"] == "ref"]
        if ref.empty:
            return group
        
        ref_score = ref[score_col].iloc[0]
        group["delta"] = group[score_col] - ref_score
        if use_abs:
            group["abs_delta"] = group["delta"].abs()
        return group
    
    df["delta"] = 0.0
    if use_abs:
        df["abs_delta"] = 0.0
    
    return df.groupby("guide_id", group_keys=False).apply(add_deltas)

Prepare top-ranked guides based on variant impact

In [ ]:
def prepare_data_by_delta(df, score_col, top_n, dataset_type):
    """Select top N guides with largest variant effects."""
    df = df.copy()
    df["guide_id"] = df.apply(lambda x: compute_guide_id(x[0], x[1], x[6]), axis=1)
    df = calculate_deltas(df, score_col)
    
    # Group guides and extract ref/alt information
    guide_data = {}
    for guide_id, group in df.groupby("guide_id", sort=False):
        ref = group[group["origin"] == "ref"]
        if ref.empty:
            continue
        
        ref_row = ref.iloc[0]
        alts = group[group["origin"] == "alt"]
        
        if score_col == "score_cfdon":
            alts = alts[alts[score_col] < ref_row[score_col]]
        
        alt_entries = [
            {
                "alt_sgRNA": alt["sgRNA_sequence"],
                "pam": alt["pam"],
                "alt_score": alt[score_col],
                "delta": alt["delta"],
                "abs_delta": alt.get("abs_delta", abs(alt["delta"])),
                SAMPLE_COL: alt[SAMPLE_COL],
                "variant_id": alt["variant_id"]
            }
            for _, alt in alts.iterrows()
        ]
        
        guide_data[guide_id] = {
            "ref_sgRNA": ref_row["sgRNA_sequence"],
            "pam": ref_row["pam"],
            "ref_score": ref_row[score_col],
            "ref_samples": ref_row[SAMPLE_COL],
            "alts": alt_entries
        }
    
    # Rank guides by worst variant effect
    rankings = []
    for gid, data in guide_data.items():
        if not data["alts"]:
            worst = 0.0
        elif score_col == "score_cfdon":
            worst = min(a["delta"] for a in data["alts"])
        else:
            worst = max(a["abs_delta"] for a in data["alts"])
        rankings.append((gid, worst))
    
    worst_df = pd.DataFrame(rankings, columns=["guide_id", "delta"]) \
        .sort_values("delta", ascending=(score_col == "score_cfdon")) \
        .reset_index(drop=True)
    
    # Include special guide if present
    if SG1617 in worst_df["guide_id"].values:
        special = worst_df[worst_df["guide_id"] == SG1617]
        others = worst_df[worst_df["guide_id"] != SG1617].head(top_n - 1)
        final_guides = pd.concat([special, others], ignore_index=True)
    else:
        final_guides = worst_df.head(top_n)
    
    final_guides["Rank"] = final_guides.index + 1
    
    # Build wide-format dataframe
    max_alts = max(len(guide_data[gid]["alts"]) for gid in final_guides["guide_id"])
    rows = []
    
    for _, row in final_guides.iterrows():
        gid = row["guide_id"]
        data = guide_data[gid]
        
        out = {
            "guide_id": gid,
            "Rank": row["Rank"],
            "ref_sgRNA": data["ref_sgRNA"],
            "pam": data["pam"],
            "ref_score": data["ref_score"],
            "ref_n_samples": data["ref_samples"],
        }
        
        for i, alt in enumerate(data["alts"]):
            prefix = f"alt{i+1}_"
            out.update({
                f"{prefix}sgRNA": alt["alt_sgRNA"],
                f"{prefix}pam": alt["pam"],
                f"{prefix}score": alt["alt_score"],
                f"{prefix}delta": alt["delta"],
                f"{prefix}abs_delta": alt["abs_delta"],
                f"{prefix}{SAMPLE_COL}": alt[SAMPLE_COL],
                f"{prefix}variant_id": alt["variant_id"],
            })
        
        # Fill missing alt columns with NaN
        for i in range(len(data["alts"]), max_alts):
            prefix = f"alt{i+1}_"
            for k in ["sgRNA", "pam", "score", "delta", "abs_delta", SAMPLE_COL, "variant_id"]:
                out[f"{prefix}{k}"] = np.nan
        
        rows.append(out)
    
    return pd.DataFrame(rows)

Utilities for plot sizing and styling

In [ ]:
def get_legend_size(n_samples, dataset_type):
    """Scale dot size based on sample count and dataset."""
    if dataset_type == "GNOMAD":
        thresholds = [1, 20, 50, 150, 500, np.inf]
    else:
        thresholds = [1, 20, 50, 100, 200, np.inf]
    
    bases = [1, 10, 35, 75, 150, 300]
    for limit, base in zip(thresholds, bases):
        if n_samples <= limit:
            return 150 * np.sqrt(base)

Create a dot plot showing variant effects across guides

In [ ]:
def dotplot_delta(df, score_col, top_n, dataset_type):
    """Plot variant effects for top N guides."""
    top_df = df.sort_values('Rank').head(top_n).copy()
    top_df['rank_chr_start'] = top_df.apply(
        lambda row: f"Rank {row['Rank']}, {row['guide_id'].split('_')[0]}:{row['guide_id'].split('_')[1]}",
        axis=1
    )
    
    plt.figure(figsize=(22, 11))
    
    # Generate color palette for variants
    alt_cols = [c for c in df.columns if c.startswith("alt") and c.endswith(f"_{SAMPLE_COL}")]
    variant_keys = list({
        row['rank_chr_start'] for _, row in top_df.iterrows()
        for col in alt_cols if pd.notna(row[col]) and row[col] != 'REF'
    })
    
    base_cmaps = ['Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                  'PuRd', 'RdPu', 'BuPu', 'GnBu', 'PuBuGn', 'BuGn',
                  'Spectral', 'coolwarm']
    palette = [sns.color_palette(cmap, 9)[7] for cmap in base_cmaps]
    
    if len(variant_keys) > len(palette):
        palette += [sns.color_palette(cmap, 9)[i] 
                    for i in range(6, 9) for cmap in base_cmaps][:len(variant_keys) - len(palette)]
    
    random.shuffle(palette)
    variant_colors = dict(zip(variant_keys, palette))
    
    # Plot alternative alleles
    for _, row in top_df.iterrows():
        for i in range(1, 1000):
            score_col_alt = f"alt{i}_score"
            samples_col_alt = f"alt{i}_{SAMPLE_COL}"
            
            if score_col_alt not in row or pd.isna(row[score_col_alt]):
                break
            
            n_samples = int(row[samples_col_alt]) if pd.notna(row[samples_col_alt]) else 1
            plt.scatter(
                row['Rank'], row[score_col_alt],
                color=variant_colors.get(row['rank_chr_start'], 'gray'),
                s=get_legend_size(n_samples, dataset_type),
                alpha=0.6, edgecolors='white', linewidth=0.5,
                marker='D' if n_samples == 1 else 'o', zorder=3
            )
    
    # Plot reference alleles
    if score_col == 'score_cfdon':
        plt.axhline(y=1, color='gray', linestyle='-', alpha=0.6, linewidth=3, zorder=5)
        ref_legend = plt.Line2D([0], [0], color='gray', linewidth=3,
                                label='Reference', alpha=0.6)
    else:
        plt.scatter(top_df['Rank'], top_df['ref_score'],
                   color='black', s=600, zorder=5,
                   edgecolors='white', linewidth=0.7, label='Reference', alpha=0.6)
        ref_legend = plt.Line2D([0], [0], marker='o', color='w', label='Reference',
                               markerfacecolor='dimgray', alpha=0.6,
                               markersize=np.sqrt(600),
                               markeredgecolor='white', linewidth=0.7)
    
    # Configure axes and labels
    ax = plt.gca()
    ax.set_xticks(top_df['Rank'])
    ax.set_xticklabels([row['ref_sgRNA'] for _, row in top_df.iterrows()],
                       rotation=45, ha='right', fontsize=15)
    
    # Bold special guides
    for label, gid in zip(ax.get_xticklabels(), top_df['guide_id'].values):
        if gid in [SG1617]:
            label.set_weight("bold")
    
    plt.yticks(fontsize=15)
    plt.xlabel('Guide', fontsize=18)
    
    ylabel = 'Variant Effect (CFD)' if score_col == 'score_cfdon' \
             else f'On-Target Efficiency ({score_col.split("_")[1].upper()})'
    title = 'Variant Effect on Alternative On-Targets' if score_col == 'score_cfdon' \
            else 'Guides On-Target Efficiency'
    
    plt.ylabel(ylabel, fontsize=19)
    plt.title(title, fontsize=28)
    
    # Create legend
    legend_labels = ['1', '2–20', '21–50', '51–100', '101–200', '>200']
    sample_counts = [1, 10, 35, 75, 150, 300]
    markers = ['D'] + ['o'] * 5
    scaled_sizes = [150 * np.sqrt(n) for n in sample_counts]
    
    size_legend_handles = [
        plt.Line2D([0], [0], marker=m, color='w', label=l,
                   markerfacecolor='gray', markersize=np.sqrt(s), alpha=0.6)
        for l, s, m in zip(legend_labels, scaled_sizes, markers)
    ]
    size_legend_handles.insert(0, ref_legend)
    
    legend = plt.legend(handles=size_legend_handles, title="Dot Size = #Samples",
                       frameon=False, bbox_to_anchor=(0.5, -0.55), loc='lower center',
                       ncol=7, fontsize=11, handletextpad=1, columnspacing=5, labelspacing=2)
    legend.get_title().set_fontsize(24)
    
    plt.ylim((-10, 110) if score_col == 'score_deepcpf1' else (-0.05, 1.05))
    plt.grid(True, alpha=0.3, linestyle='--')
    sns.despine()
    plt.subplots_adjust(bottom=0.25)
    plt.show()

Once defined the functions required to plot variants impact on guides efficiency
and on-target activity, we focus again on BCL11A +58 Erythroid enhancer. For these
analysis we combine the candidate guides using the variants from 1000G, HGDP, and
gnomAD datasets.

In [ ]:
# combine data from different datasets
for df in [bcl11a_1000G, bcl11a_HGDP, bcl11a_GNOMAD]:
    if "n_samples" not in df.columns:
        df["n_samples"] = df["n_ref"] if "n_ref" in df.columns else np.nan

bcl11a_ALL = pd.concat([
    bcl11a_1000G.assign(dataset="1000G"),
    bcl11a_HGDP.assign(dataset="HGDP"),
    bcl11a_GNOMAD.assign(dataset="GNOMAD")
], ignore_index=True)

dotplot_delta(
    prepare_data_by_delta(bcl11a_ALL, "score_azimuth", 25, "ALL"),
    "score_azimuth",
    25,
    "ALL",
)
dotplot_delta(
    prepare_data_by_delta(bcl11a_ALL, "score_cfdon", 25, "ALL"),
    "score_cfdon",
    25,
    "ALL",
)